In [ ]:
from io import BytesIO
import hashlib
import random

import util
from test_framework.key import ECKey, ECPubKey, SECP256K1_ORDER, generate_schnorr_nonce
from test_framework.musig import generate_musig_key, aggregate_schnorr_nonces, sign_musig, aggregate_musig_signatures
from test_framework.script import  CScript, OP_1, OP_CHECKSIG, TaprootSignatureHash, TapLeaf, TapTree, Node
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness, ser_string
from test_framework.address import program_to_witness

# 2.2 TapTweak

* Part 1 - Tweaking the Public Key
* Part 2 - Commitment Schemes with Tweaks
* Part 3 - TapTweak Script Commitments


## Part 1: Tweaking the Public Key

The script commitment scheme of taproot is based on tweaking the public key. 

* `[01] [33B Tweaked Public Key]`

Tweaking a public key means to alter it with a value (the tweak) so that it remains spendable with knowledge of the original private key and tweak. 

* Output Script: `[01] [P + T]` = `[01] [xG + tG]`
* Spendable by: `x + t`
* `P`: Internal Key
* `t`: TapTweak
* `T`: TapTweak Point


Importantly, an observer cannot distinguish between a tweaked and untweaked public key.

### Example 1.1 - Signing with a tweaked keypair

* A) Key pair generation: `xG = P`
* B) A tweak is positive scalar value where: `t < SECP256K1 Order` 
* C) A tweak has a corresponding point: `T = t*G`
* D) The private key is tweaked by the tweak scalar: `x' = x + t`
* E) The public key is tweaked by the tweak point: `P' = P + T`
* F) Tweaked keypair `(x', P')` will produce a valid signature.

In [ ]:
# A) Generate Key Pair.
privatekey = ECKey()
privatekey.generate()
publickey = privatekey.get_pubkey()

# B) Tweak: t < SECP256K1 Order
tweak = hashlib.sha256(b'tweak').digest()
assert(int.from_bytes(tweak, 'big') < SECP256K1_ORDER)

# C) Tweak point.
tweak_private = ECKey()
tweak_private.set(tweak, True)
tweak_point = tweak_private.get_pubkey()

# D) Derive tweaked private key.
privatekey_tweaked = privatekey + tweak_private

# E) Derive public key point.
publickey_tweaked = publickey + tweak_point

# F) Sign message with tweaked key pair and verify signature.
msg = hashlib.sha256(b'msg').digest()
sig = privatekey_tweaked.sign_schnorr(msg)
print(publickey_tweaked.verify_schnorr(sig, msg))

### Programming Exercise 1.2 -  Signing with a tweaked 2-of-2 Musig key pairs.

* A. 2-of-2 Musig key generation.
    * Key pair generation for all participants: `xG = P`
    * Aggregate all public keys: `pk_musig`
    * Apply challenge factors to all private keys: `x' = c * x`
* B. Tweak the `pk_musig`.
* C. Signing
    * Generate indidividual nonces & aggregate: `R_agg`
    * Sign individually & aggregate: `sig_agg`
    * _Question: Which participant(s) need to tweak keys?_


In [ ]:
# A1) Generate Key Pairs.


# A2) Aggregate all public keys: 


# A3) Apply challenge factors to keys.

    
# B) Tweak musig public key.
# Given: Tweak.
tweak = hashlib.sha256(b'tweak').digest()


# C1) Nonce generation & aggregation.


# C2) Signing and signature aggregation.
msg = hashlib.sha256(b'msg').digest()



## Part 2: Commitment Schemes with Tweaks

Taproot uses the tweak as a commitment for spending script paths. However, simply applying the committed value as a public key tweak is not sufficient, as this does not represent a cryptographic commitment.

![test](images/taptweak0.jpg)

Instead, the committed value must first be hashed with the untweaked public key point. 

**This prevents modification of both untweaked secret and tweak for a given pubkey point Q.**

### Example 2.1- Modifying the tweak for a tweaked public key Q.

* A) Tweaking the public key to obtain: `Point Q`
* B) Create a new tweak for the same point Q: `t'`
* C) Solve for `x'` so that `x'G + t'G = Q`

In [ ]:
# A) Tweaking the public key.
x = ECKey()
x.generate()
pk = x.get_pubkey()
t = hashlib.sha256(b'tweak').digest()
t_int = int.from_bytes(t, "big")
Q = pk.tweak_add(t)

# B) Modifying the tweak.
t2 = hashlib.sha256(b'tweak2').digest()
t2_int = int.from_bytes(t2, "big")

# C) Solving for: x` = x - (t' - t)
x_int = int.from_bytes(x.get_bytes(),"big")
s = (t2_int + (SECP256K1_ORDER - t_int)% SECP256K1_ORDER) % SECP256K1_ORDER
x2_int = (x_int + (SECP256K1_ORDER - s)% SECP256K1_ORDER) % SECP256K1_ORDER

print((x2_int + t2_int)% SECP256K1_ORDER == (x_int + t_int)% SECP256K1_ORDER)

### Example 2.2 - Tweaking Pubkey with `H(P|msg)`

* A) Key pair generation: 
* B) The tweak is the hash of both P and message: **`t = H(P|msg)`**
* C) A tweak has a corresponding point: `T = t*G`
    * A private key is tweaked by the tweak scalar: `x' = x + t`
    * The public key can be tweaked by the tweak point: `P' = P + T`
* D) We can verify that signature with tweaked key pair is valid.

In [ ]:
# A) Key pair generation.
privkey = ECKey()
privkey.generate()
publickey = privkey.get_pubkey()

# B) Compute tweak from H(P|msg)
# Note: Taproot/Taptweak actualy uses tagged hashes (See Below).
tag = "TapTweak"
ss = hashlib.sha256(tag.encode('utf-8')).digest()
ss += ss
ss += hashlib.sha256(b'commitment').digest()
t = hashlib.sha256(ss).digest()

# C) Determine Tweak point.
tweak_private = ECKey()
tweak_private.set(t, True)
tweak_point = tweak_private.get_pubkey()

privkey_tweaked = privkey + tweak_private
publickey_tweaked = publickey + tweak_point

# F) Sign message and verify signature.
msg = hashlib.sha256(b'msg').digest()
sig = privkey_tweaked.sign_schnorr(msg)
print(publickey_tweaked.verify_schnorr(sig, msg))

### Taproot: Tagged Hashes.

The Taproot Proposal describes tagged hashes for both the taptree and taptweak. 

The TapTweak uses this double nested hashing function because it already used in TapBranches and TapLeafs, where it provides context uniqueness across the Bitcoin protocol. The 64B length of the two sha256 digest concatenation  lends itself to optimization in implementations. 

**Tagged Hash:**
* `tagged_hash` = `sha256(sha256("Tag") + sha256("Tag") + data)`


**TapTree Node Hashes:**
* `TapTweak` 
    * `= sha256(sha256("TapTweak") + sha256("TapTweak") + node_hash)`
* `TapBranch` 
    * `= sha256(sha256("TapBranch") + sha256("TapBranch") + node_hash_left|node_hash_right)`
* `TapLeaf` 
    * `= sha256(sha256("TapLeaf") + sha256("TapLeaf") + node_hash_left|node_hash_right)`
    
_Ordering of left and right node hash data is determined lexicographically, see next section._

## Part 3 - TapTweak Script Commitments

The TapTweak commits a Taptree to the segwit version 1 public key. It does so with a commitment structure resembling familiar merkle tree construction.

_Please note that the taptree uses tagged hashes which prevent node height ambiguity currently found in the transaction merkle tree, which allow an attacker to create a node which can be reinterpreted as either a leaf or internal node._


**The TapTree is different than the header merkle tree in the following ways:**

* Tapleafs can be located at different heights.
* Ordering of TapLeafs is determined lexicograpically.
* Location of nodes are tagged (No ambiguity of node type).
 
![test](images/taptweak1.jpg)


### Programming Exercise 3.1 - Constructing a TapTweak from TapScripts.

In the cell below, we will commit three pay-to-pubkey scripts to a taptweak and then derive the segwit address which can be spent by fulfilling these scriptpaths and the internal. We will use the same merkle tree structure as in the previous illustration.

* 1. Compute TapLeafs A, B and C.
* 2. Compute Internal node TapBranch AB.
* 3. Compute TapTweak
* 4. Derive the segwit output address.


In [ ]:
TAPSCRIPT_VER = bytes([0xc0]) # See TapScript chapter for more details.
internal_pubkey = ECPubKey()
internal_pubkey.set(bytes.fromhex('03af455f4989d122e9185f8c351dbaecd13adca3eef8a9d38ef8ffed6867e342e3'))

# Derive pay-to-pubkey scripts.
secA = ECKey()
secB = ECKey()
secC = ECKey()
secA.generate()
secB.generate()
secC.generate()
pkA = secA.get_pubkey()
pkB = secA.get_pubkey()
pkC = secA.get_pubkey()
scriptA = CScript([pkA.get_bytes(), OP_CHECKSIG])
scriptB = CScript([pkB.get_bytes(), OP_CHECKSIG])
scriptC = CScript([pkC.get_bytes(), OP_CHECKSIG])

# Method: Returns Tagged Hash.
def tagged_hash(tag, input_data):
    data = hashlib.sha256(tag.encode('utf-8')).digest()
    data += data
    data += input_data
    return hashlib.sha256(data).digest()

# Method: Returns TapBranch hash.
def tapbranch(taggedhash_left, taggedhash_right):
    if taggedhash_left > taggedhash_right:
        taggedhash_left, taggedhash_right = taggedhash_right, taggedhash_left
    return tagged_hash("TapBranch", taggedhash_left + taggedhash_right)  

# 1) Compute TapLeafs A, B and C.
# Note: ser_string(data) is a function which adds compactsize to input data.


# 2) Compute Internal node TapBranch AB.


# 3) Compute TapTweak.


# 4) Derive the segwit output address.



**Run the cell below to check if you have correctly computed the TapTweak.**

In [ ]:
# This code uses the TapTree and TapLeaf classes to construct the same tweak as above

tapleafA = TapLeaf()
tapleafB = TapLeaf()
tapleafC = TapLeaf()
taptree = TapTree()

tapleafA.from_script(scriptA)
tapleafB.from_script(scriptB)
tapleafC.from_script(scriptC)

tapbranchAB = Node()
tapbranchAB.left = tapleafA
tapbranchAB.right = tapleafB

tapbranchABC = Node()
tapbranchABC.left = tapbranchAB
tapbranchABC.right = tapleafC

taptree.root = tapbranchABC
taptree.key = internal_pubkey

script_v1, tweak, c_map = taptree.construct()
print("TapTweak:", tweak.hex())

### Programming Exercise 3.2 - Spending a Taproot Output along the Key Path.

Now that we have understood how public keys can be tweaked to commit pay-to-pubkey scripts to a Segwit version 1 output, let us spend this output along the key path. Such as spend does not reveal the script commitments to the observer and is indistinguishable any other key path spend.

* 1. Construct taproot output with tweaked public key.
* 2. Send funds from the wallet to the taproot output (Segwit Address).
* 3. Create and sign transaction which sends funds back to wallet with the tweaked private key.

**Note: Most of the cells have been populated already. Simply tweak the pubkey and signing key.**

**Start TestNodes**

In [ ]:
# Start TestNodes.
test = util.TestWrapper()
test.setup()

**Generate Wallet Balance**

In [ ]:
# Generate Coins for Bitcoin Node Wallet.
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print(balance)

**1) Construct taproot output with tweaked public key.**

In [ ]:
# Create Tweaked Pubkey Output.
sec = ECKey()
sec.generate()
internal_pubkey = sec.get_pubkey()

# Taptweak from example 2.3.1
taptweak = bytes.fromhex('2a2fb476ec9962f262ff358800db0e7364287340db73e5e48db36d1c9f374e30')

# TODO:
# taproot_pubkey = 


# TODO: 
# segwit_address =


**2) Send funds from the wallet to the taproot output (Segwit Address).**

In [ ]:
# Send funds to taproot output.
txid = test.nodes[0].sendtoaddress(segwit_address, balance / 100000)
print("Funding tx:", txid)

# Deserialize wallet transaction.
tx = CTransaction()
tx_hex = test.nodes[0].getrawtransaction(txid)
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# Determine Output Index of Segwit V1 Output.
# (Wallet places change output at a random txout index.)
outputs = iter(tx.vout)
output = next(outputs)
index = 0

while (output.scriptPubKey != CScript([OP_1, taproot_pubkey_v1])):
    output = next(outputs)
    index += 1
output_value = output.nValue

**3) Spend taproot output with key path.**

In [ ]:
tx_schnorr = CTransaction()
tx_schnorr.nVersion = 1
tx_schnorr.nLockTime = 0
outpoint = COutPoint(tx.sha256, index)
tx_schnorr_in = CTxIn(outpoint = outpoint)
tx_schnorr.vin = [tx_schnorr_in]

# Generate new Bitcoin Core wallet address to send funds back to.
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])

# Determine minimum fee required for mempool acceptance.
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet.
dest_output= CTxOut(nValue=output_value-min_fee, scriptPubKey=scriptpubkey)
tx_schnorr.vout = [dest_output]

# Sign transaction with tweaked private key.
# TODO: 
# sig =


# Construct transaction witness.
witness = CScriptWitness()
witness.stack.append(sig)
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
tx_schnorr.wit.vtxinwit.append(witness_in)

# Serialize Schnorr transaction for broadcast.
tx_schnorr_str = tx_schnorr.serialize().hex()

# Test mempool acceptance.
print(test.nodes[0].testmempoolaccept([tx_schnorr_str]))


**Shutdown TestNodes**

In [ ]:
# Shutdown TestNodes.
test.shutdown()